In [ ]:
# Adding system path
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))
# sys.path

In [ ]:
# Set to show warnings only once
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setting up displays
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from dash import Dash,html,dcc,Input,Output
app = Dash(__name__)
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

In [ ]:
from dash import Dash,html,dcc

In [ ]:
# ##center allign all the figure outputs.
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# from IPython.display import display, HTML
# from plotly.graph_objs import *
# import numpy as np
# init_notebook_mode(connected=True)

# display(HTML("""
# <style>
# .output {
#     display: flex;
#     align-items: center;
#     text-align: center;
# }
# </style>
# """))

In [ ]:
# Setting up autoreload for libs
%load_ext autoreload
%autoreload 2
%aimport optiml.queries

In [ ]:
# Initialize connection to Snowflake and set analysis date
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/kiva')
# Initialize query library
from optiml.queries import SNFLKQuery
qlib = SNFLKQuery(connection, 'KIV', cache_dir)
sdate = '2022-10-01'
edate = '2022-10-31'
print(f"The analysis is carried our for date range {sdate} to {edate}")

# User Analysis

In [ ]:
## This would be my starting point for users that are a security risk
idle_user_start_date = '2022-04-01'
idle_user_end_date = '2022-10-31'
df = qlib.idle_users(idle_user_start_date, idle_user_end_date)
df.head()

In [ ]:
# import plotly.express as px
# data_canada = px.data.gapminder().query("country == 'Canada'")
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['last_success_login'], y=[0]*len(df['last_success_login'].to_list()), mode='markers', marker_size=20
))
fig.update_xaxes(showgrid=False, title_text="last sucess login")
fig.update_yaxes(showgrid=False, 
                 zeroline=True, zerolinecolor='black', zerolinewidth=3,
                 showticklabels=False)
fig.update_layout(height=200, plot_bgcolor='white', title_text="Idle Users")
fig.show()

In [ ]:
## Users never logged in but not deleted
idle_user_start_date = '2022-04-01'
idle_user_end_date = '2022-10-31'
df = qlib.users_never_logged_in(idle_user_start_date, idle_user_end_date)
df.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['created_on'], y=[0]*len(df['created_on'].to_list()), mode='markers', marker_size=20
))
fig.update_xaxes(showgrid=False, title_text="Created on")
fig.update_yaxes(showgrid=False, 
                 zeroline=True, zerolinecolor='black', zerolinewidth=3,
                 showticklabels=False)
fig.update_layout(height=200, plot_bgcolor='white', title_text="Never logged in")
fig.show()

In [ ]:
df=qlib.users_full_table_scans(start_date='2018-04-01', end_date='2022-10-31')
df.head()

In [ ]:
df=qlib.heavy_users(start_date="2019-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.idle_roles(start_date="2021-01-01", end_date="2022-02-02")
df.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['created_on'], y=[0]*len(df['created_on'].to_list()), mode='markers', marker_size=20
))
fig.update_xaxes(showgrid=False, title_text="Created on")
fig.update_yaxes(showgrid=False, 
                 zeroline=True, zerolinecolor='black', zerolinewidth=3,
                 showticklabels=False)
fig.update_layout(height=200, plot_bgcolor='white', title_text="Idle roles")
fig.show()

In [ ]:
## Warehouse credit usage over time



app = Dash(__name__)



fig = px.scatter(df_expensiveq, x=df_expensiveq["query_id"], y=df_expensiveq["relative_performance_cost"],
                 color=df_expensiveq["user_name"], hover_name=df_expensiveq["warehouse_size"],size_max=30)
fig.update_traces(marker_size=20)


app.layout = html.Div([
    dcc.Graph(
        id='life-exp-vs-gdp',
        figure=fig
    )
])

fig.show()

In [ ]:
app = Dash(__name__)



app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['user_name'].unique(),
                'User_name',
                id='xaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='xaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='yaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},

    )
])


@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                     y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                     hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')

    return fig